# Case Study Unit 4 - Signal Frontier Analysis
### Cory Adams, Chris Boomhower, Alexandra Fisher, Alex Frye
### *MSDS 7333, September 23, 2017*

## Abstract
Abstract here

## Introduction
Intro here

## Literature Review
Lit review here

## Background
Background here

In [ ]:
## Load necessary modules
import pandas_datareader as web
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

## Methods
Methods here

## Results
Results here

***Need to decide upon dates as team***

In [ ]:
names = ['TXN', 'ON', 'NXPI', 'DIOD', 'MXIM']
def get_px(stock, start, end):
    return web.DataReader(stock, 'yahoo', start, end)

px = pd.DataFrame({n:get_px(n, None, None)['Adj Close'] for n in names})
px

In [ ]:
px = px.asfreq('B').fillna(method='pad')
rets = px.pct_change()
((1 + rets).cumprod() - 1).plot()

In [ ]:
def calc_mom(price, lookback, lag):
    mom_ret = price.shift(lag).pct_change(lookback)
    ranks = mom_ret.rank(axis=1, ascending=False)
    demeaned = ranks.subtract(ranks.mean(axis=1), axis=0)
    return demeaned.divide(demeaned.std(axis=1), axis=0)

In [ ]:
compound = lambda x : (1 + x).prod() - 1
daily_sr = lambda x: x.mean() / x.std()

def strat_sr(prices, lb, hold):
    # Compute portfolio weights
    freq = '%dB' % hold
    port = calc_mom(prices, lb, lag=1)

    daily_rets = prices.pct_change()

    # Compute portfolio returns
    port = port.shift(1).resample(freq, how='first')
    returns = daily_rets.resample(freq, how=compound)
    port_rets = (port * returns).sum(axis=1)

    return daily_sr(port_rets) * np.sqrt(252 / hold)

In [ ]:
strat_sr(px, 70, 30)

In [ ]:
lookbacks = range(20, 90, 5)
holdings = range(20, 90, 5)
dd = defaultdict(dict)
for lb in lookbacks:
    for hold in holdings:
        dd[lb][hold] = strat_sr(px, lb, hold)

ddf = pd.DataFrame(dd)
ddf.index.name = 'Holding Period'
ddf.columns.name = 'Lookback Period'

In [ ]:
import matplotlib.pyplot as plt

def heatmap(df, cmap=plt.cm.gray_r):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    axim = ax.imshow(df.values, cmap=cmap, interpolation='nearest')
    ax.set_xlabel(df.columns.name)
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns))
    ax.set_ylabel(df.index.name)
    ax.set_yticks(np.arange(len(df.index)))
    ax.set_yticklabels(list(df.index))
    plt.colorbar(axim)

In [ ]:
heatmap(ddf)

## Discussion and Future Work
Discussion, conclusions and future work here

## References
References, if any, listed here